<a href="https://colab.research.google.com/github/EmmanuelABonsu/CE888/blob/main/src/project/evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.metrics import confusion_matrix

In [2]:
model = keras.models.load_model('drive/MyDrive/trained_model/fire_detection_model_run_2')

In [3]:
test_path = "drive/MyDrive/CE888_data/Test"

In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=(254, 254),
        shuffle = False,
        class_mode='binary',
        batch_size=1)

Found 8617 images belonging to 2 classes.


In [5]:
score = model.evaluate(test_generator,               
                                  max_queue_size=10,                 
                                  workers=0,                         
                                  use_multiprocessing=False,         
                                  verbose=0)
print("loss: %.3f - acc: %.3f" % (score[0], score[1]))


loss: 2.537 - acc: 0.722


In [6]:
predictions = model.predict(test_generator,
                                    max_queue_size=10,
                                    workers=0,
                                    use_multiprocessing=False,
                                    verbose=0)

# Evaluate predictions
predictedClass = np.argmax(predictions, axis=1)
trueClass = test_generator.classes[test_generator.index_array]
classLabels = list(test_generator.class_indices.keys())

# Create confusion matrix
confusionMatrix = (confusion_matrix(
  y_true=trueClass,                                       
  y_pred=predictedClass))                                 
print(confusionMatrix)

[[3987 1150]
 [1248 2232]]
